# Mortgage Document Classification

Mortgage companies often need to process large volumes of diverse document types to extract business-critical data. Today, lenders are faced with the challenge of managing a manual, slow, and expensive process to extract data and insights from documents.

To help address the issue, Amazon Textract Analyze Lending improves business process efficiency through automation and accuracy, reducing loan processing costs and providing the ability to scale quickly based on changing demand.


## Analyze Lending
Analyze Lending is a managed intelligent document processing API that fully automates the classification and extraction of information from loan packages. Customers simply upload their mortgage loan documents to the Analyze Lending API and its pre-trained machine learning models will automatically classify and split by document type, and extract critical fields of information from a mortgage loan packet.


In this workshop, we will use Analyze Lending for Document Classification in the intelligent document processing workflow.

At the beginning of our document processing stage, it may not be obvious as do which documents are present in the mortgage packet. Using Amazon Textract Analyze Lending we will first identify these documents into their respective classes. Once we know which documents are present in the packet, we can run any kind of validation such as look for missing/required documents, extract specific document in a specific way such as ID documents and so on. The figure below explains this process.
  

<p align="center">
  <img src="./images/classification.png" alt="cfn" width="800px"/>
</p>

## Setup Notebook

In [ ]:
!python -m pip install -q amazon-textract-response-parser amazon-textract-caller amazon-textract-prettyprinter boto3 --upgrade

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
awscli 1.34.16 requires botocore==1.35.16, but you have botocore 1.35.28 which is incompatible.


In [ ]:
import boto3
from textractcaller import call_textract_lending
from textractprettyprinter.t_pretty_print import convert_lending_from_trp2
from IPython.display import Image, display, HTML, JSON, IFrame
import trp.trp2_lending as tl
import json
import sagemaker
import botocore
import os
import pandas as pd
from sagemaker import get_execution_role

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/base_serializers.py:28: UserWarning: A NumPy version >=1.23.5 and <2.3.0 is required for this version of SciPy (detected version 1.22.4)
  import scipy.sparse


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


## Data preparation

The sample lending package `lending_package.pdf` . For this workshop, we will be a PDF formatted package, that includes the following documents:

1. Pay Slip
2. Check
3. ID Document
4. Tax form 1099-DIV
5. Bank Statement
6. Tax form W2
7. Homeowners Insurance Application Form



In [ ]:
document = 'lending_package.pdf'

In [ ]:
# variables
data_bucket = sagemaker.Session().default_bucket()
region = boto3.session.Session().region_name
account_id = boto3.client('sts').get_caller_identity().get('Account')

os.environ["BUCKET"] = data_bucket
os.environ["REGION"] = region
role = sagemaker.get_execution_role()

print(f"SageMaker role is: {role}\nDefault SageMaker Bucket: s3://{data_bucket}")

s3=boto3.client('s3')
textract = boto3.client('textract', region_name=region)

SageMaker role is: arn:aws:iam::842675974990:role/service-role/AmazonSageMaker-ExecutionRole-20240919T022017
Default SageMaker Bucket: s3://sagemaker-us-east-1-842675974990


## Upload sample data to S3 bucket

The sample package has multiple documents in one file (lending_package.pdf). We will upload this to an S3 bucket for processing.

In [ ]:
# Upload images to S3 bucket:
bucket = sagemaker.Session().default_bucket()
print(f"SageMaker Bucket: s3://{data_bucket}")

!aws s3 cp docs/lending_package.pdf s3://{data_bucket}/idp/textract/ --only-show-errors

input_file = 's3://' + bucket + '/idp/textract' + '/' + document
print(f"Lending Package uploaded to S3: {input_file}")

SageMaker Bucket: s3://sagemaker-us-east-1-842675974990
Lending Package uploaded to S3: s3://sagemaker-us-east-1-842675974990/idp/textract/lending_package.pdf


## NOTE: ACTION IS REQUIRED OUTSIDE OF THE NOTEBOOK FOR THE NEXT CODE CELL TO WORK

You need to grant your SageMaker-Execution-Role Textract access for the next line to work.

The easiest way to do this is via the AWS IAM Console. Select your SageMakerExexcutionRole and add Textract full access.

## Process the document with Textract Analyze Lending
We will process the lending package using the Textract Caller helper library, and then print out details for how the pages were classified.

For more information, see [Textract Caller helper library.](https://pypi.org/project/amazon-textract-caller/)

The format of the response JSON returned is described in the Textract Analyze Lenging documentation here: [Link](https://docs.aws.amazon.com/textract/latest/dg/lending-document-classification-extraction.html)


In [ ]:
textract_json = call_textract_lending(input_document=input_file, boto3_textract_client=textract)

In [ ]:
print('Lending Package processed')
print('Pages: {}'.format(textract_json['DocumentMetadata']['Pages']))


Lending Package processed
Pages: 7


We will next print the classifications detected for each page of the package
Type – The normalized value associated with a detection.


In [ ]:
results = textract_json['Results']

for page in results:
    print("Page Number: {}".format(page["Page"]), "Page Classification: {}".format(page["PageClassification"]["PageType"]))

Page Number: 1 Page Classification: [{'Value': 'PAYSLIPS', 'Confidence': 99.99993896484375}]
Page Number: 2 Page Classification: [{'Value': 'CHECKS', 'Confidence': 99.98819732666016}]
Page Number: 3 Page Classification: [{'Value': 'IDENTITY_DOCUMENT', 'Confidence': 99.9999771118164}]
Page Number: 4 Page Classification: [{'Value': '1099_DIV', 'Confidence': 99.99998474121094}]
Page Number: 5 Page Classification: [{'Value': 'BANK_STATEMENT', 'Confidence': 97.54032135009766}]
Page Number: 6 Page Classification: [{'Value': 'W_2', 'Confidence': 99.99996185302734}]
Page Number: 7 Page Classification: [{'Value': 'UNCLASSIFIED', 'Confidence': 99.99408721923828}]


----

### How does Textract Analyze Lending classify the document pages?

Let's take a look at the full JSON response from AnalyzeLending. The `results` variable consists of a list with 7 elements for each of the 7 pages in the `lending_package.pdf` document.

In [ ]:
display(JSON(results, expanded=False))

<IPython.core.display.JSON object>

Let's look at the JSON for the first page in the document.

In [ ]:
display(JSON(results[0], expanded=True))

<IPython.core.display.JSON object>

For page 1 of the document, Textract Analyze Lending classified the page as a `PAYSLIP`. On the payslip, it will look for key payslip fields. For Example:
- Start Date: `___` (as there is none on this payslip)
- End date: 7/18/2008
- Pay Date: 7/25/2008
- Borrower Name: John Stiles

![analyze-lending-payslip-header.png](attachment:analyze-lending-payslip-header.png)



Note that the last page is an unsupported document type, a Homeowners Insurance Application Form. In module 2, we will process the fields extracted from each of the documents.

Let's next store the JSON output to a file. A file named "lending-doc-output.json" will be created locally on this SageMaker Studio instance, in the `aws-ai-intelligent-document-processing directory/industry/mortgage` directory.
We will use the same JSON file for our extractions in the next module.

In [ ]:
json.dump(textract_json, open("lending-doc-output.json", "w"))

---
## Use the Textract Response Library to convert the response to an easier format to work with

We are calling the [Textract Response Parser Library](https://pypi.org/project/amazon-textract-response-parser/) and then convert the Textract Analyze Lending JSON response to a flattened array for CSV export.


The convert_lending_from_trp2() method returns a list of `[{page_classification}_{page_number_within_document_type}, page_number_in_document, key, key_confidence, value, value_confidence, key-bounding-box.top, key-bounding-box.height, key-bb.width, key-bb.left, value-bounding-box.top, value-bb.height, value-bb.width, value-bb.left]`

In [ ]:
trp2_doc: tl.TFullLendingDocument = tl.TFullLendingDocumentSchema().load(textract_json)
lending_array = convert_lending_from_trp2(trp2_doc)

A file named "lending-doc-output.csv" will be created locally on this SageMaker Studio instance, `aws-ai-intelligent-document-processing directory/industry/mortgage` directory.
Open the file and use the  CSV display to view each element extracted.
You can also download the file to view locally on your laptop in your favorite spreadsheet application.

In [ ]:
import csv

index_fields = ['{page_classification}_{page_number_within_document_type}', 'page_number_in_document', 'key','key_confidence','value','value_confidence','key-bounding-box.top','key-bounding-box.height','key-bb.width','key-bb.left','value-bounding-box.top','value-bb.height','value-bb.width','value-bb.left']

with open('analyze-lending-output.csv', 'w') as f:
    csv_writer = csv.writer(f)
    csv_writer.writerow(index_fields)
    csv_writer.writerows(lending_array)


Open the CSV file (use the panel on the left, in the `mortgage` directory) and observe the fields extracted, for example:

![analyze-lending-payslip-csv-sample.png](attachment:analyze-lending-payslip-csv-sample.png)

# Assignment Question #1

In this notebook, we processes several different types of documents, such as:

1. Pay Slip
2. Check
3. ID Document
4. Tax form 1099-DIV
5. Bank Statement
6. Tax form W2

What are the full test of document/page-types that are supported by Textract call_textract_lending?

(add your answer in the markdown cell that follows)

The API specifically targets the mortgage and financial services industry. Specifically a stack of predefined document types that are to do with lending. The full list is shown below of document/page-type supported by call_textract_lending API

Application Form (URLA - Uniform Residential Loan Application)
Appraisal
Asset Statement (Bank Statement)
Check
Credit Report
Deed of Trust
Escrow Letter
Flood Certificate
Gift Letter
Hazard Insurance Policy
HUD-1 Settlement Statement
ID Document (e.g., Driver's License, Passport)
Income Statement (Pay Slip, Pay Stub)
Lease Agreement
Letter of Explanation
Loan Estimate
Mortgage Insurance Application
Note
Other Asset Document (various other statements not covered specifically)
Other Income Document (e.g., letters confirming income sources)
Other Letter
Pay Slip
Power of Attorney
Purchase Contract
Settlement Statement (Closing Disclosure)
Tax Form (e.g., W2, 1099-DIV, 1040, etc.)
Title Policy
Verification of Employment
Verification of Mortgage
Verification of Rent

# Assignment Question #2

For an identified document-type (aka page-type), if a corresponding value is not found for one or more of the associated fields during the processing of call_textract_lending, what value(s) is output for the field in returned JSON document? Providing an example would be ideal.


(add your answer in the markdown cell that follows)

The returned JSON shows missing values in three ways:

1. Empty String (""): For fields where a value is expected but not found, Textract returns an empty string as the field value.
2. "undetected" Value: In some cases, especially with classification or if certain fields cannot be detected with confidence, the field value might be set to "undetected", indicating that Textract did not detect the field or its content reliably.
3. Null Value: Occasionally, a null value might also be used, particularly when the field itself is missing entirely from the extracted data.



In [ ]:
#Example of code below. Where start date field is not able to be found

In [ ]:
{
  "DocumentMetadata": {
    "Pages": 1
  },
  "Results": [
    {
      "Page": 1,
      "PageClassification": {
        "PageType": [{"Value": "PAYSLIPS", "Confidence": 99.9999}]
      },
      "Fields": [
        {
          "Key": "PAY_PERIOD_START_DATE",
          "Value": "",
          "KeyConfidence": 99.0,
          "ValueConfidence": 0.0
        },
        {
          "Key": "PAY_PERIOD_END_DATE",
          "Value": "7/18/2008",
          "KeyConfidence": 99.0,
          "ValueConfidence": 98.0
        }
      ]
    }
  ]
}
